# PREPROCESSING IMAGE DATA TO LINKED DATA
In this notebook, we evaluate the image assets in a session repository.
For every jpeg/png with its accompanying xmp or xmp file, an IMAGENODE metadata class is created 
that governs all the metadata of the image (paths, pose, etc.).
As ouput, the method generates RDF GRAPHS (.ttl) files

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [14]:
from rdflib import Graph, plugin
import uuid    
import importlib
import PIL.Image as PILimage
import os.path
import cv2
#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

In [15]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
%autoreload 2

## 1. INITIALIZE SESSION

In [17]:
## INPUTS
projectPath= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"tests")#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = os.path.join(projectPath , "Sample3") #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon
graphPath= os.path.join(sessionPath,"imageGraph.ttl") 

## 2. READ PRESENT JPEG/PNG/XMP/XML CONTAINING IMAGES + METADATA
An IMAGENODE is created per image in the session

In [18]:
# path=sessionPath+"\\100_0848_0005.JPG"
# img=cv2.imread(path)
# graphPath=sessionPath+"\\imgGraph.ttl"
# xmpPath=sessionPath+"\\IMG_2173.xmp"
# node1=ImageNode(path=path, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)
# node2=ImageNode(img=img, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)
# # node3=ImageNode(graphPath=graphPath, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)
# # node3=ImageNode(xmpPath=xmpPath, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)


In [19]:
# print(node1.subject)
# print(node1.geospatialTransform)

In [20]:
# path='C:\\RepoHeinder\\geomapi\\tests\\Sample3\\101_0366_0036.jpg'
# subject='101_0366_0036'
# fileformat=['.jpg','.png']
# print(ut.get_extension(path))
# print(subject)
# print(path)
# if ut.get_extension(path) in fileformat:
#     print('wowowo')
# if subject in path:
#     print('jippie')

In [27]:
files=[f for f in ut.get_list_of_files(r'D:\geomapi\tests\testfiles\mesh') if f.endswith('.obj') or f.endswith('.ply')]
                            
nodes=[]
for f in files:
    nodes.append(MeshNode(name=ut.get_filename(f),path=f,getResource=True))
    
print(len(nodes))

tl.nodes_to_graph(nodes,graphPath=r'D:\geomapi\tests\testfiles\graphs\mesh_graph.ttl',save=True)

2


<Graph identifier=N57a7d6bf32a54cb38599aa2b4b3cd6e7 (<class 'rdflib.graph.Graph'>)>

'e57'

In [47]:
import laspy as las

files=ut.get_list_of_files(r'D:\geomapi\tests\testfiles\pcd')                            
nodes=[]
for f in files:
    if not f.endswith('.las'):
        print(f)
        nodes.append(PointCloudNode(name=ut.get_filename(f),path=f,getResource=True))
    else:
        print(f)
        p=las.open(f)
        myLas = p.read()
        nodes.append(PointCloudNode(resource=gmu.las_to_pcd(myLas)))
    
print(len(nodes))

tl.nodes_to_graph(nodes,graphPath=r'D:\geomapi\tests\testfiles\graphs\pcd_graph.ttl',save=True)

D:/geomapi/tests/testfiles/pcd/lidar.e57
D:/geomapi/tests/testfiles/pcd/lidar.xml
D:/geomapi/tests/testfiles/pcd/parking.e57
D:/geomapi/tests/testfiles/pcd/parking.las
[[48.01851653 46.839283    3.83217479]
 [48.98833464 47.26428604  5.44773863]
 [46.60432055 46.14498518  6.71159314]
 ...
 [56.73801802 50.37362672  2.01952029]
 [56.75973131 50.33730697  1.93189763]
 [56.81056597 50.3355751   1.90233363]]
D:/geomapi/tests/testfiles/pcd/parking.pcd
5


<Graph identifier=N42b9a867c414487dafec814b561c52f5 (<class 'rdflib.graph.Graph'>)>

In [21]:
path=r'D:\geomapi\tests\testfiles\img\road.xml'
nodes=tl.img_xml_to_nodes(path)

print(len(nodes))

78


In [22]:
graphPath=r'D:\geomapi\tests\testfiles\graphs\road_img_graph.ttl'
tl.nodes_to_graph(nodes,graphPath=graphPath,save=True)

<Graph identifier=N5157f150f0774fdebb44fce07e21e848 (<class 'rdflib.graph.Graph'>)>

In [8]:
import xml.etree.ElementTree as ET 
import numpy as np
from rdflib import URIRef
# Retrieve all image files in the session along with their xml metadata
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
nodelist=[]

for path in allSessionFilePaths:        
    
    if path.endswith(".JPG") or path.endswith(".PNG"): 
        testXmpPath=path.replace('JPG','xmp')
        if testXmpPath in allSessionFilePaths:
            nodelist.append(ImageNode(path=path,xmpPath=testXmpPath,sessionPath=sessionPath))
        else:
            nodelist.append(ImageNode(path=path,sessionPath=sessionPath))

    if path.endswith(".xml"):
        nodelist.extend(tl.img_xml_to_nodes(path, getResource=True,graphPath=graphPath))
          
print(str(len(nodelist))+' ImageNodes are created.')
# print(str(exifCounter)+' have poses in exif data')
# print(str(xmpCounter)+' have poses in xmp data')
# print(str(xmlCounter)+' have poses in xml data (not implemented yet)')
print('Note that these nodes do not yet contain actual data but only the metadata')

21 ImageNodes are created.
Note that these nodes do not yet contain actual data but only the metadata


## 3. CREATE RDF GRAPH FOR All IMAGENODES AND EXPORT THEM TO .TTL
An RDF graph and .ttl file is created for all images in the session
(images themselves are not stored in the graph, only metadata)

In [9]:
graph=tl.nodes_to_graph(nodelist,graphPath=graphPath)
subjectList=[1 for node in nodelist if node.graph is not None]
graph.serialize(destination=graphPath)
print (str(len(subjectList))+' Nodes succesfully serialized in: '+ graphPath)

21 Nodes succesfully serialized in: c:\RepoHeinder\geomapi\tests\Sample3\imageGraph.ttl


In [10]:
print(graph.serialize())

@prefix e57: <http://libe57.org#> .
@prefix exif: <http://www.w3.org/2003/12/exif/ns#> .
@prefix gom: <https://w3id.org/gom#> .
@prefix openlabel: <https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#> .
@prefix v4d: <https://w3id.org/v4d/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<file:///c:/RepoHeinder/geomapi/tests/Sample3/101_0366_0036> a v4d:ImageNode ;
    e57:cartesianTransform """[[-9.01077021e-01 -4.33659085e-01  0.00000000e+00  6.00575993e+05]
 [ 4.33659085e-01 -9.01077021e-01  0.00000000e+00  6.96281654e+05]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  3.13845765e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]""" ;
    e57:geospatialTransform "[51.07460836115559, 3.6635813333635228, 74.11900086895052]" ;
    exif:imageHeight 3648 ;
    exif:imageWidth 5472 ;
    exif:resolutionUnit 2 ;
    gom:coordinateSystem "geospatial-wgs84" ;
    v4d:name "101_0366_0036" ;
    v4d:path "10